In [1]:
%matplotlib inline

In [ ]:
from subprocess import call
call(["pip", "install", "nltk"])
call(["pip", "install", "textblob"])
call(["pip", "install", "IDisplay"])

In [ ]:
import nltk

nltk.download()

In [ ]:
import IDisplay

# Scrape swear word list

We scrape swear words from the web from the site:
http://www.noswearing.com/

It is a community driven list of swear words.

In [4]:
import string
import os
import requests
from fake_useragent import UserAgent
from lxml import html

def requests_get(url):        
    ua = UserAgent().random  
    return requests.get(url, headers={'User-Agent': ua})

def get_swear_words(save_file='swear_words.txt'): 
    """
    Scrapes a comprehensive list of swear words from noswearing.com
    """
    words = ['niggas']
    if os.path.isfile(save_file):
        with open(save_file, 'rt') as f:
            for line in f:
                words.append(line.strip())
        
        return words
        
    base_url = 'http://www.noswearing.com/dictionary/'
    letters = '1' + string.ascii_lowercase
    
    for letter in letters:
        full_url = base_url + letter
        result = requests_get(full_url)
        tree = html.fromstring(result.text)
        search = tree.xpath("//td[@valign='top']/a[@name and string-length(@name) != 0]")
        
        if search is None:
            continue
        
        for result in search:
            words.append(result.get('name').lower())
            
    with open(save_file, 'wt') as f:
        for word in words:
            f.write(word)
            f.write('\n')
    
    return words

print(get_swear_words())


['niggas', 'anus', 'arse', 'arsehole', 'ass', 'ass-hat', 'ass-jabber', 'ass-pirate', 'assbag', 'assbandit', 'assbanger', 'assbite', 'assclown', 'asscock', 'asscracker', 'asses', 'assface', 'assfuck', 'assfucker', 'assgoblin', 'asshat', 'asshead', 'asshole', 'asshopper', 'assjacker', 'asslick', 'asslicker', 'assmonkey', 'assmunch', 'assmuncher', 'assnigger', 'asspirate', 'assshit', 'assshole', 'asssucker', 'asswad', 'asswipe', 'axwound', 'bampot', 'bastard', 'beaner', 'bitch', 'bitchass', 'bitches', 'bitchtits', 'bitchy', 'blow job', 'blowjob', 'bollocks', 'bollox', 'boner', 'brotherfucker', 'bullshit', 'bumblefuck', 'butt plug', 'butt-pirate', 'buttfucka', 'buttfucker', 'camel toe', 'carpetmuncher', 'chesticle', 'chinc', 'chink', 'choad', 'chode', 'clit', 'clitface', 'clitfuck', 'clusterfuck', 'cock', 'cockass', 'cockbite', 'cockburger', 'cockface', 'cockfucker', 'cockhead', 'cockjockey', 'cockknoker', 'cockmaster', 'cockmongler', 'cockmongruel', 'cockmonkey', 'cockmuncher', 'cocknose'

# Testing TextBlob

I don't really like TextBlob as it tries to be "nice", but lacks a lot of basic functionality.

1. Stop words not included
2. Tokenizer is pretty meh.
3. No built in way to obtain word frequency

In [ ]:
import os
import operator

import pandas as pd
from textblob import TextBlob, WordList
from nltk.corpus import stopwords

def get_data_paths():
    dir_path = os.path.dirname(os.path.realpath('.'))
    data_dir = os.path.join(dir_path, 'billboard-hot-100-data')
    dirs = [os.path.join(data_dir, d, 'songs.csv') for d in os.listdir(data_dir) 
            if os.path.isdir(os.path.join(data_dir, d))]
    
    return dirs

def lyric_file_to_text_blob(row):
    """
    Appends lyrics column to data frame given that it has the 'lyrics_path' column.
    """
    text_blob = None
    
    with open(row['lyrics_path'], 'r') as f:
        text_blob = TextBlob(f.read())
        
    return text_blob

def remove_stop_words(word_list):
    wl = WordList([])
    
    stop_words = stopwords.words('english')
    for word in word_list:
        if word.lower() not in stop_words:
            wl.append(word)
    
    return wl

def word_freq(words, sort='desc'):
    """
    Returns frequency table for all words provided in the list.
    """
    
    reverse = sort == 'desc'
    
    freq = {}
    for word in words:
        if word in freq:
            freq[word] = freq[word] + 1
        else:
            freq[word] = 1
            
    return sorted(freq.items(), key=operator.itemgetter(1), reverse=reverse)

data_paths = get_data_paths()
songs = pd.read_csv(data_paths[0])
songs["lyrics"] = songs.apply(lyric_file_to_text_blob, axis=1)

all_words = WordList([])

for i, row in songs.iterrows():
    all_words.extend(row['lyrics'].words)

cleaned_all_words = remove_stop_words(all_words)
cleaned_all_words = pd.DataFrame(word_freq(cleaned_all_words.lower()), columns=['word', 'frequency'])
cleaned_all_words

In [ ]:
import pandas as pd
import nltk

def lyric_file_to_text(row):
    """
    Adds lyrics to given panda data frame as a string. Must have lyrics_path.
    """
    text = None
    
    with open(row['lyrics_path'], 'r') as f:
        text = f.read()
        
    return text

def remove_extra_junk(word_list):
    words = []
    remove = [",", "n't", "'m", ")", "(", "'s", "'", "]", "["]
    
    for word in word_list:
        if word not in remove:
            words.append(word)
            
    return words
    
    
data_paths = get_data_paths()
songs = pd.read_csv(data_paths[0])
songs["lyrics"] = songs.apply(lyric_file_to_text, axis=1)

all_words = []

for i, row in songs.iterrows():
    all_words.extend(nltk.tokenize.word_tokenize(row['lyrics']))

cleaned_all_words = [w.lower() for w in remove_extra_junk(remove_stop_words(all_words))]
freq_dist = nltk.FreqDist(cleaned_all_words)

freq_dist.plot(50)
freq_dist.most_common(100)
#cleaned_all_words = pd.DataFrame(word_freq(cleaned_all_words), columns=['word', 'frequency'])
#cleaned_all_words

# Repetitive songs skewing data?

Some songs may be super reptitive. Lets look at a couple of songs that have the word in the title. These songs probably repeat the title a decent amount in their song. Hence treating all lyrics as one group of text less reliable in analyzing frequency.

To simplify this process, we can look at only single word titles. This will at least give us a general idea if the data could be skewed by a single song or not.

In [ ]:
for i, song in songs.iterrows():
    title = song['title']
    title_words = title.split(' ')
    
    if len(title_words) > 1:
        continue
    
    lyrics = song['lyrics']
    words = nltk.tokenize.word_tokenize(lyrics)
    clean_words = [w.lower() for w in remove_extra_junk(remove_stop_words(words))]
    
    dist = nltk.FreqDist(clean_words)
    freq = dist.freq(title_words[0].lower())
    
    if freq > .1:
        print(song['artist'], title)

## Seems pretty reptitive

There are a handful of single word song titles that repeat the title within the song at least 10% of the time. This gives us a general idea that there is most likely a skew to the data. I think it is safe to assume that if a single word is repeated many times, the song is most likely reptitive.

Lets look at the song "water" by Ugly God to confirm.

In [ ]:
song_title_to_analyze = 'Water'

lyrics = songs['lyrics'].where(songs['title'] == song_title_to_analyze, '').max()
print(lyrics)
words = nltk.tokenize.word_tokenize(lyrics)
clean_words = [w.lower() for w in remove_extra_junk(remove_stop_words(words))]
water_dist = nltk.FreqDist(clean_words)
water_dist.plot(25)

water_dist.freq(song_title_to_analyze.lower())

## Looking at swear word distribution

Let's look at the distribution of swear words...

In [ ]:
sws = []

for sw in set(get_swear_words()):
    sws.append({'word': sw,
                'dist': freq_dist.freq(sw)})
    
sw_df = pd.DataFrame.from_dict(sws)
sw_df.nlargest(10, 'dist').plot(x='word', kind='bar')